# 训练模型测试

In [3]:
import os
from torchvision import transforms 
import torch.nn as nn
import torch
from tqdm import tqdm_notebook as tqdm
from Dataset import YTBDatasetCNN
from Network import CNNNet

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] ='2' # 设置跑第几个GPU
# 使用cuda运算
device=torch.device("cuda")

### 初始化数据集

In [ ]:
dataset = YTBDatasetCNN(root_dir='../aligned_images_DB', img_size=224)
dataload = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=16, num_workers=4)

### 初始化萌新（模型）

In [ ]:
model=CNNNet().to(device)
model = model.train()
# if os.path.exists("model.acc.pth"):
#     model.load_state_dict(torch.load("model.acc.pth"))

### 查看可以更新的参数

In [ ]:
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name)

### 读取存储好的CNN模型权值

In [ ]:
 model.load_state_dict(torch.load("./checkpoints/cnn_modelacc0.9982.pth"))

# 训练

In [ ]:
# model.init_weights()
acc_max = 0
# optimizer = torch.optim.Adadelta(model.parameters(),lr=0.05)
optimizer =torch.optim.RMSprop(model.parameters(),lr=0.01)
# optimizer=torch.optim.Adagrad(model.parameters(), lr=0.0005, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
for epoch in range(300):
    total_loss = 0
    total_acc = 0
    total_size = 0
    bar = tqdm(dataload)
    for i, (x, y, _) in enumerate(bar):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        # label=y.unsqueeze(1)
        # In your for loop
        # label_one_hot.scatter_(1, label, 1)
        loss, acc_num = model.process(x, y)
        total_size += x.size(0)
        loss.backward()
        optimizer.step()
        # b=pred.item()
        total_acc += acc_num
        total_loss += loss.item()

        bar.set_postfix(loss=f"{total_loss/(i+1):0.4f}",
                        acc=f"{total_acc/total_size:0.4f}",
                        epoch=f"{epoch+1}")
    acc = total_acc / total_size
    torch.save(model.state_dict(), "cnn_model.pth")
    if acc_max < acc:
        acc_max = max(total_acc / total_size, acc_max)
        torch.save(model.state_dict(), f"./checkpoints/cnn_model_acc{acc_max:0.4f}.pth")
#     if acc>0.8:
#         optimizer = torch.optim.Adadelta(model.parameters(),lr=1-acc)

# 新方法的训练(RBG_diffenrence)

In [ ]:
from tqdm import tqdm_notebook as tqdm

import os
import torch
from Dataset import  YTBDatasetCNN_RGBDiff
from Network import  CNN_RGBDiff
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # 设置跑第几个GPU
device = torch.device("cuda")

In [ ]:
batch_size = 16
dataset = YTBDatasetCNN_RGBDiff(csv_file='../splits.txt', root_dir='../aligned_images_DB', img_size=224)
num_class=int(len(dataset)/dataset.person_frames)
dataload = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=2)
model = CNN_RGBDiff(class_num=num_class).to(device)
len(dataset)

In [ ]:
model.load_state_dict(torch.load("./checkpoints/cnn_RGBdiffer_model_acc0.9494.pth"))

In [ ]:
model = model.train()
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

        acc_max = 0

In [ ]:
# optimizer = torch.optim.RMSprop(model.parameters(), lr=0.5,weight_decay=1e-4)
optimizer=torch.optim.RMSprop(model.parameters(), lr=1e-3, weight_decay=1e-9)   
# label_one_hot = torch.LongTensor(batch_size,  num_class).to(device)
# label_one_hot.zero_()
for epoch in range(300):
    total_loss = 0
    total_acc = 0
    total_size = 0
    bar = tqdm(dataload)
    for i, (x, d, y) in enumerate(bar):
        optimizer.zero_grad()
        x,d, y = x.to(device),d.to(device), y.to(device)

        x=torch.cat([x,d],dim=1)
        # label=y.unsqueeze(1)
        # In your for loop
        # label_one_hot.scatter_(1, label, 1)
        loss, acc_num = model.process(x, y)
        total_size += x.size(0)
        loss.backward()
        optimizer.step()
        # b=pred.item()
        total_acc += acc_num
        total_loss += loss.item()

        bar.set_postfix(loss=f"{total_loss/(i+1):0.4f}",
                        acc=f"{total_acc/total_size:0.4f}",
                        epoch=f"{epoch+1}")
    acc = total_acc / total_size
    torch.save(model.state_dict(), "cnn_RGBdiffer_model.pth")
    if acc > 0.8:
        torch.optim.RMSprop(model.parameters(), lr=1-acc, weight_decay=1e-3)
    if acc >0.9:
        torch.optim.RMSprop(model.parameters(), lr=1e-2, weight_decay=1e-4)    
    if acc >0.95:
        torch.optim.RMSprop(model.parameters(), lr=1e-4, weight_decay=1e-7)
    if acc_max < acc:
        acc_max = max(total_acc / total_size, acc_max)
        torch.save(model.state_dict(), f"./checkpoints/cnn_RGBdiffer_model_acc{acc_max:0.4f}.pth")

### 换CNN的训练

In [5]:
from tqdm import tqdm_notebook as tqdm
import os
import torch
from Dataset import  YTBDatasetCNN
from Network import  CNNNet
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # 设置跑第几个GPU 
device = torch.device("cuda")

In [6]:
batch_size = 32
dataset = YTBDatasetCNN(csv_file='../splits.txt', root_dir='../aligned_images_DB', img_size=224)
num_class=int(len(dataset)/dataset.person_frames)
dataload = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=2)
model = CNNNet(class_num=num_class).to(device)
len(dataset)

638000

### 改变cnn

In [7]:
import torchvision.models as models
model.cnn_model =  models.densenet161(num_classes=128).to(device)
# models 里面有很多CNN模型可以无脑调用， 就在这里稍微改一下

In [8]:
model.load_state_dict(torch.load("./checkpoints/densenet_cnn_acc0.9966.pth"))

In [9]:
model = model.train()
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

        acc_max = 0

cnn_model.features.conv0.weight
cnn_model.features.norm0.weight
cnn_model.features.norm0.bias
cnn_model.features.denseblock1.denselayer1.norm1.weight
cnn_model.features.denseblock1.denselayer1.norm1.bias
cnn_model.features.denseblock1.denselayer1.conv1.weight
cnn_model.features.denseblock1.denselayer1.norm2.weight
cnn_model.features.denseblock1.denselayer1.norm2.bias
cnn_model.features.denseblock1.denselayer1.conv2.weight
cnn_model.features.denseblock1.denselayer2.norm1.weight
cnn_model.features.denseblock1.denselayer2.norm1.bias
cnn_model.features.denseblock1.denselayer2.conv1.weight
cnn_model.features.denseblock1.denselayer2.norm2.weight
cnn_model.features.denseblock1.denselayer2.norm2.bias
cnn_model.features.denseblock1.denselayer2.conv2.weight
cnn_model.features.denseblock1.denselayer3.norm1.weight
cnn_model.features.denseblock1.denselayer3.norm1.bias
cnn_model.features.denseblock1.denselayer3.conv1.weight
cnn_model.features.denseblock1.denselayer3.norm2.weight
cnn_model.features.d

In [ ]:
# model.init_weights()
acc_max = 0
# optimizer = torch.optim.Adadelta(model.parameters(),lr=0.05)
optimizer =torch.optim.RMSprop(model.parameters(),lr=0.005)
# optimizer=torch.optim.Adagrad(model.parameters(), lr=0.0005, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
for epoch in range(300):
    total_loss = 0
    total_acc = 0
    total_size = 0
    bar = tqdm(dataload)
    for i, (x, y, _) in enumerate(bar):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        # label=y.unsqueeze(1)
        # In your for loop
        # label_one_hot.scatter_(1, label, 1)
        loss, acc_num = model.process(x, y)
        total_size += x.size(0)
        loss.backward()
        optimizer.step()
        # b=pred.item()
        total_acc += acc_num
        total_loss += loss.item()

        bar.set_postfix(loss=f"{total_loss/(i+1):0.4f}",
                        acc=f"{total_acc/total_size:0.4f}",
                        epoch=f"{epoch+1}")
    acc = total_acc / total_size
    torch.save(model.state_dict(), "densenet_cnn.pth")
    if acc_max < acc:
        acc_max = max(total_acc / total_size, acc_max)
        torch.save(model.state_dict(), f"./checkpoints/densenet_cnn_acc{acc_max:0.4f}.pth")
#     if acc>0.8:
#         optimizer = torch.optim.Adadelta(model.parameters(),lr=1-acc)